In [11]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, Sequential

## Basic Res Block

In [13]:
class BasicBlock(layers.Layer):
    # 残差模块
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        # 第一个卷积单元
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        # 第二个卷积单元
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()

        if stride != 1:# 通过1x1卷积完成shape匹配
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:# shape匹配，直接短接
            self.downsample = lambda x:x

    def call(self, inputs, training=None):

        # [b, h, w, c]，通过第一个卷积单元
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)
        # 通过第二个卷积单元
        out = self.conv2(out)
        out = self.bn2(out)
        # 通过identity模块
        identity = self.downsample(inputs)
        # 2条路径输出直接相加
        output = layers.add([out, identity])
        output = tf.nn.relu(output) # 激活函数

        return output


## class `ResNet`

In [14]:
class ResNet(keras.Model):
    # 通用的ResNet实现类
    def __init__(self, layer_dims, num_classes=10): # [2, 2, 2, 2]
        super(ResNet, self).__init__()
        # 根网络，预处理
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                                ])
        # 堆叠4个Block，每个block包含了多个BasicBlock,设置步长不一样
        self.layer1 = self.build_resblock(64,  layer_dims[0])
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)

        # 通过Pooling层将高宽降低为1x1
        self.avgpool = layers.GlobalAveragePooling2D()
        # 最后连接一个全连接层分类
        self.fc = layers.Dense(num_classes)

    def call(self, inputs, training=None):
        # 通过根网络
        x = self.stem(inputs)
        # 一次通过4个模块
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # 通过池化层
        x = self.avgpool(x)
        # 通过全连接层
        x = self.fc(x)

        return x



    def build_resblock(self, filter_num, blocks, stride=1):
        # 辅助函数，堆叠filter_num个BasicBlock
        res_blocks = Sequential()
        # 只有第一个BasicBlock的步长可能不为1，实现下采样
        res_blocks.add(BasicBlock(filter_num, stride))

        for _ in range(1, blocks):#其他BasicBlock步长都为1
            res_blocks.add(BasicBlock(filter_num, stride=1))

        return res_blocks


## define `resnet18` and `resnet34`

In [15]:
def resnet18():
    return ResNet([2, 2, 2, 2])

def resnet34():
    return ResNet([3, 4, 6, 3])

## Train CIFAR10

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, datasets, Sequential
import os


def preprocess(x, y):
    # [0, 1]
    x = 2*tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)
    return x, y

(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)
print(x.shape, y.shape, x_test.shape, y_test.shape)

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(1000).map(preprocess).batch(512)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(512)


def main():
    # [b, 32, 32, 3] => [b, 1, 1, 512]
    model = resnet18()
    model.build(input_shape=(None, 32, 32, 3))
    model.summary()

    optimizer = optimizers.Adam(lr=1e-4)    

    for epoch in range(100):
        for step, (x, y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                # [b, 32, 32, 3] => [b, 10]
                logits = model(x)

                # [b] => [b, 10]
                y_onehot = tf.one_hot(y, depth=10)
                # compute loss
                loss = tf.losses.categorical_crossentropy(
                    y_onehot, logits, from_logits=True)
                loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            if step % 50 == 0:
                print(epoch, step, "loss: ", float(loss))

        # test
        total_num = 0
        total_correct = 0
        for x, y in test_db:
            logits = model(x)
            prob = tf.nn.softmax(logits, axis=1)
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)

            total_num += x.shape[0]
            total_correct += int(correct)

        acc = total_correct / total_num
        print(epoch, " acc: ", acc)


main()


(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
Model: "res_net_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_40 (Sequential)  (None, 30, 30, 64)        2048      
                                                                 
 sequential_41 (Sequential)  (None, 30, 30, 64)        148736    
                                                                 
 sequential_42 (Sequential)  (None, 15, 15, 128)       526976    
                                                                 
 sequential_44 (Sequential)  (None, 8, 8, 256)         2102528   
                                                                 
 sequential_46 (Sequential)  (None, 4, 4, 512)         8399360   
                                                                 
 global_average_pooling2d_5   multiple                 0         
 (GlobalAveragePooling2D)                                        
 

/home/yeting/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


0 0 loss:  2.307429313659668
0 50 loss:  1.7627296447753906
0  acc:  0.42
1 0 loss:  1.4981410503387451
1 50 loss:  1.4778732061386108
1  acc:  0.4823
2 0 loss:  1.3367785215377808
2 50 loss:  1.263134241104126
2  acc:  0.5418
3 0 loss:  1.2479180097579956
3 50 loss:  1.1265352964401245
3  acc:  0.5848
4 0 loss:  1.0675344467163086
4 50 loss:  1.0797522068023682
4  acc:  0.6123
5 0 loss:  1.0224738121032715
5 50 loss:  0.9543542265892029
5  acc:  0.647
6 0 loss:  0.9391425848007202
6 50 loss:  0.9529329538345337
6  acc:  0.6537
7 0 loss:  0.9515607357025146
7 50 loss:  0.8349283337593079
7  acc:  0.6498
8 0 loss:  0.8794112205505371
8 50 loss:  0.7401345372200012
8  acc:  0.6731
9 0 loss:  0.8470720052719116
9 50 loss:  0.793770432472229
9  acc:  0.6957
10 0 loss:  0.7347293496131897
10 50 loss:  0.7342371940612793
10  acc:  0.7066
11 0 loss:  0.6459107995033264
11 50 loss:  0.6197052001953125
11  acc:  0.7167
12 0 loss:  0.5965052247047424
12 50 loss:  0.617832601070404
12  acc:  0.72